In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import r2_score
from xgboost import XGBRegressor
import warnings
warnings.filterwarnings("ignore")




In [2]:
df=pd.read_csv('cleaned_house_rent_dataset.csv')

In [3]:
df

,BHK,Rent,Size,Bathroom,Current Floor,Total Floors,Area Type_Built Area,Area Type_Carpet Area,Area Type_Super Area,City_Bangalore,...,City_Delhi,City_Hyderabad,City_Kolkata,City_Mumbai,Furnishing Status_Furnished,Furnishing Status_Semi-Furnished,Furnishing Status_Unfurnished,Tenant Preferred_Bachelors,Tenant Preferred_Bachelors/Family,Tenant Preferred_Family
0,2,10000,1100,2,0.0,2.0,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
1,2,20000,800,1,1.0,3.0,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
2,2,17000,1000,1,1.0,3.0,0,0,1,0,...,0,0,1,0,0,1,0,0,1,0
3,2,10000,800,1,1.0,2.0,0,0,1,0,...,0,0,1,0,0,0,1,0,1,0
4,2,7500,850,1,1.0,2.0,0,1,0,0,...,0,0,1,0,0,0,1,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4741,2,15000,1000,2,3.0,5.0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
4742,3,29000,2000,3,1.0,4.0,0,0,1,0,...,0,1,0,0,0,1,0,0,1,0
4743,3,35000,1750,3,3.0,5.0,0,1,0,0,...,0,1,0,0,0,1,0,0,1,0
4744,3,45000,1500,2,23.0,34.0,0,1,0,0,...,0,1,0,0,0,1,0,0,0,1


In [4]:
print(df.isnull().sum())

BHK                                   0
Rent                                  0
Size                                  0
Bathroom                              0
Current Floor                        38
Total Floors                         38
Area Type_Built Area                  0
Area Type_Carpet Area                 0
Area Type_Super Area                  0
City_Bangalore                        0
City_Chennai                          0
City_Delhi                            0
City_Hyderabad                        0
City_Kolkata                          0
City_Mumbai                           0
Furnishing Status_Furnished           0
Furnishing Status_Semi-Furnished      0
Furnishing Status_Unfurnished         0
Tenant Preferred_Bachelors            0
Tenant Preferred_Bachelors/Family     0
Tenant Preferred_Family               0
dtype: int64


In [5]:
df['Current Floor'].fillna(df['Current Floor'].median(), inplace=True)
df['Total Floors'].fillna(df['Total Floors'].median(), inplace=True)


In [6]:
print(df.isnull().sum())

BHK                                  0
Rent                                 0
Size                                 0
Bathroom                             0
Current Floor                        0
Total Floors                         0
Area Type_Built Area                 0
Area Type_Carpet Area                0
Area Type_Super Area                 0
City_Bangalore                       0
City_Chennai                         0
City_Delhi                           0
City_Hyderabad                       0
City_Kolkata                         0
City_Mumbai                          0
Furnishing Status_Furnished          0
Furnishing Status_Semi-Furnished     0
Furnishing Status_Unfurnished        0
Tenant Preferred_Bachelors           0
Tenant Preferred_Bachelors/Family    0
Tenant Preferred_Family              0
dtype: int64


In [7]:
X = df.drop("Rent", axis=1)   # Rent is the target variable
y = df["Rent"]

In [8]:
# Train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
#Define model
xgb = XGBRegressor()

#Define parameters grid
param_grid = {
    'n_estimators': [100, 200],
    'learning_rate': [0.05, 0.1],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1],
    'colsample_bytree': [0.8, 1]
}


In [10]:
#Setup GridSearchCV
grid_search = GridSearchCV(estimator=xgb, param_grid=param_grid, cv=3, scoring='r2', verbose=1, n_jobs=-1)

#Fit the model
grid_search.fit(X_train, y_train)

#Best model
best_model = grid_search.best_estimator_


Fitting 3 folds for each of 48 candidates, totalling 144 fits


In [11]:
#Predict on test set
y_pred = best_model.predict(X_test)

#R² Score
r2 = r2_score(y_test, y_pred)
print(f"R² Score of best XGBoost model: {r2:.4f}")

R² Score of best XGBoost model: 0.6184


In [12]:
import pickle
filename="finalized_model_xgboost_house_rent_model.sav"
pickle.dump(grid_search,open(filename,'wb'))

In [13]:
#Load the model 
loaded_model = pickle.load(open(filename, 'rb'))

